# 1. Fields as Curried Functions

In a 1994 paper, Gary T Leavens explained, in a wonderfully graspable manner, that Fields in physics can actually be viewed as curried functions. If you understand what curried functions are, then you already have a good idea of what fields are—these sort of bridges are incredibly helpful to those of us peering in from both sides of the divide (for me, from the CS side). The examples were in scheme and so the syntax inversion might make it a bit difficult for the unpracticed (such as myself) to follow, hence my translating (and hopefully, helpful to you too). My examples here are in F#, where units of measure are especially helpful in making things clearer.

##Table of Contents
* [1. Fields as Curried Functions](#1.-Fields-as-Curried-Functions)
* [2. Gravity](#2.-Gravity)
	* [2.1 Graphing the near Earth Gravitational Pull](#2.1-Graphing-the-near-Earth-Gravitational-Pull)
	* [2.2 Not Falling is Unnatural](#2.2-Not-Falling-is-Unnatural)
* [3. Vector Fields](#3.-Vector-Fields)
	* [3.1 Finally, On Currying vs Fields](#3.1-Finally,-On-Currying-vs-Fields)
		* [3.1.1 A quick note on Observables](#3.1.1-A-quick-note-on-Observables)
		* [3.1.2 Quantum Field Theory](#3.1.2-Quantum-Field-Theory)
		* [3.1.3 Speculations or, Here be Dragons](#3.1.3-Speculations-or,-Here-be-Dragons)
* [4. Examples by Visualizations of and Interacting with Static Fields](#4.-Examples-by-Visualizations-of-and-Interacting-with-Static-Fields)
	* [4.1 Generalizing to Static Fields](#4.1-Generalizing-to-Static-Fields)


# 2. Gravity

First some definitions:

In [1]:
//HIDDEN open system, funscript include
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2013\open\funscript\FunScript.dll"
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2013\open\funscript\FunScript.Interop.dll"
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2013\open\funscript\packages\FunScript.TypeScript.Binding.lib\lib\net40\FunScript.TypeScript.Binding.lib.dll"
open FunScript
open FunScript.TypeScript
open System

[<FunScript.JS>]
let konst x _ = x

let inline roundn (n:int) x = Math.Round(float x, n)

let inline (</) x f = f x
 
let inline (/>) f y = (fun x -> f x y)  


val konst ∈ x∈'a ⟹ 'b ⟹ 'a
val inline roundn ∈
  n∈int ⟹ x∈ ^a ⟹ float
    when  ^a ∈ (static member op_Explicit ∈  ^a ⟹ float)
val inline ( </ ) ∈ x∈'a ⟹ f∈('a ⟹ 'b) ⟹ 'b
val inline ( /> ) ∈ f∈('a ⟹ 'b ⟹ 'c) ⟹ y∈'b ⟹ x∈'a ⟹ 'c


In [272]:
//HIDDEN
let inline joinToStringWith sep (s:'a seq) = String.Join(sep, s)
let third (_,_,c) = c
let tohtmlTableString headers d = 
    let entryFormat horD xs = xs |> Seq.map (fun x -> sprintf "<t%s>%s</t%s>" horD x horD) |> joinToStringWith "\n"
    let els = 
      d 
      |> Seq.map (entryFormat "d" >> fun s -> "<tr>" + s + "</tr>") 
      |> joinToStringWith "\n"
    sprintf "<table>%s%s</table>" (headers |> entryFormat "h") els
      
let tohtmlTable headers d = 
   {Html = tohtmlTableString headers d}
   
let embedScript s = {Html = sprintf """<script>%s</script>""" s}

let asHtml s = {Html = s}


val inline joinToStringWith ∈ sep∈string ⟹ s∈seq<'a> ⟹ string
val third ∈ 'a ⨯ 'b ⨯ c∈'c ⟹ 'c
val tohtmlTableString ∈ headers∈seq<string> ⟹ d∈seq<#seq<string>> ⟹ string
val tohtmlTable ∈ headers∈seq<string> ⟹ d∈seq<#seq<string>> ⟹ HtmlOutput
val embedScript ∈ s∈string ⟹ HtmlOutput
val asHtml ∈ s∈string ⟹ HtmlOutput


In [3]:
[<Measure>] type kg
[<Measure>] type m
[<Measure>] type s
[<Measure>] type N = kg * m / s^2

let G = 6.674e-11<N * m^2 / kg^2>


val G ∈ float<N m²/kg²> = 6.674e-11


In [4]:
let grav_force (m1:float< kg>) (r:float<m>) (m2:float<kg>) =        
    if r = 0.<m> then 0.<N> 
    else ((m1 * m2) * G) / (r * r)


val grav_force ∈ m1∈float<kg> ⟹ r∈float<m> ⟹ m2∈float<kg> ⟹ float<N>


This is a curried function since it's a function that takes a value and returns a function specialized to the passed in value.

In [15]:
//HIDDEN asides
let asideHtm s = {Html = sprintf "<div style='border-radius:0.25em;background-color:#FFD954;color:#7F3300;border: 1px solid;font-size:110%%;padding:0.2em 0.6em 0.3em'>%s</div>" s}


val asideHtm ∈ s∈string ⟹ HtmlOutput


In [178]:
//HIDDENINPUT
asideHtm "<b>Briefly</b>: a curried function such as 
plus: (+) ∈ (int ⟹ int ⟹ int), allows us to say, pre-apply 5 to it. As in, e.g., f = (+) 5. 
Then we can do f 3 = 8 or f 10 = 15 etc. Basically, with the preapplication of 5 to (+), we get back a 
new function: int ⟹ int which takes a number and returns 5 + that number. Easy yes?"

Briefly : a curried function such as 
plus: (+) ∈ (int ⟹ int ⟹ int), allows us to say, pre-apply 5 to it. As in, e.g., f = (+) 5. 
Then we can do f 3 = 8 or f 10 = 15 etc. Basically, with the preapplication of 5 to (+), we get back a 
new function: int ⟹ int which takes a number and returns 5 + that number. Easy yes?

Returning to our function `grav_force`, when we put it into F# we got the type: 

`val grav_force ∈ m1∈float<kg> ⟹ r∈float<m> ⟹ m2∈float<kg> ⟹ float<N>`

So everything looks good, it takes a mass (such as Earth) and returns a function that takes a distance (e.g. at the surface) which itself returns a function: one from masses (your mass) to Forces (your weight). This function is in fact the definition of a (_scalar_) gravitational field but more on that later. We can _specialize_ the function and get earth's field.

In [5]:
let earth_mass = 5.96e24<kg>
let earth_rad = 6.37e6<m>
let earth_grav = grav_force earth_mass


val earth_mass ∈ float<kg> = 5.96e+24
val earth_rad ∈ float<m> = 6370000.0
val earth_grav ∈ (float<m> ⟹ float<kg> ⟹ float<N>)


Similarly, we can specialize to the surface (by applying a distance to `earth_grav`) or we can compute the mass of an object at a particular distance by passing in a distance _and_ a mass. For example, noticing that `N = kg m/s²`, we can compute Earth's accelaration at surface simply by passing in _1kg_ and the Earth's radius for a value of 9.802877992 N (which is just m/s² for this case). But we can do more interesting things with curried functions. We can flip the field equation to compute the gravitational force at a list of distances from earth's surface (a bit more flexible than how physics is typically taught isn't it):

In [6]:
let flip f a b = f b a
let flipped_field = flip earth_grav 1.<kg>


val flip ∈ f∈('a ⟹ 'b ⟹ 'c) ⟹ a∈'b ⟹ b∈'a ⟹ 'c
val flipped_field ∈ (float<m> ⟹ float<N>)


In [181]:
[for dist in 0.0<m>..1e5<m>..1e6<m> -> 6.37e6<m> + dist] |> List.map flipped_field


val it ∈ float<N> list =
  [9.802877992; 9.502194172; 9.215135446; 8.940890874; 8.678708962; 8.42789251;
   8.187793968; 7.957811259; 7.737383994; 7.525990059; 7.323142521]

The map applies our function to each distance in the list to get a list of forces. So at a distance of 1000km, the gravitational acceleration is ~7.32 m/sec². 

In [182]:
//HIDDENINPUT
asideHtm "<b>Trivia</b>: Mount Everest's height is 8848 meters, g = 9.776 m/sec² there—only 0.28% 
weaker than on the surface."

Trivia : Mount Everest's height is 8848 meters, g = 9.776 m/sec² there—only 0.28% 
weaker than on the surface.

Below you can look at the gravitational acceleration on the surface of the moon and Mars. The moon has a mere acceleration of 1.62 m/s² and Mars is only just over 2x that—it is for this reason that many are concerned over the health effects (on skeletomuscular integrity) of an extended mission to Mars. 

Additionally, we can look at the Earth/Moon system. The Moon and the Earth both exert a force of $2 \times10^{20}\,N$ on each other. Both are forever falling towards each other, with the condition that—on the odd chance that—if ever they should meet, things would not end well for both of them. A tragic love story if ever there was one.

In [6]:
let mass_moon = 7.3459e22<kg>
let mass_mars = 6.41693e23<kg>
let rad_moon = 1737.5<m> * 1000.
let radius_mars = 3386.<m> * 1000.

let mean_moon_dist = 3.85e8<m>

grav_force mass_moon rad_moon    1.<kg>  , 
grav_force mass_mars radius_mars 1.<kg>  ,
grav_force earth_mass mean_moon_dist mass_moon


val mass_moon ∈ float<kg> = 7.3459e+22
val mass_mars ∈ float<kg> = 6.41693e+23
val rad_moon ∈ float<m> = 1737500.0
val radius_mars ∈ float<m> = 3386000.0
val mean_moon_dist ∈ float<m> = 385000000.0
val it ∈ float<N> ⨯ float<N> ⨯ float<N> =
  (1.623983408, 3.735421349, 1.971314948e+20)

## 2.1 Graphing the near Earth Gravitational Pull

To make things a bit more concrete than a list of numbers, I graph some commonly known locations and the gravitational acceleration felt there. Looking at these, I can't help but think of the common conception (held by myself for a long time too) that space is "out there" and far away; when really, space (LEO, at least) is literally within walking distance. Most satellites, the International Space Station included, aren't floating in space; they're still deep within Earth's gravitational well!

In [8]:
let pois = [|"~Low Earth Orbit Min",   160.<m>  
             "~Low Earth Orbit Max",  2000.<m>  
             "GEO Synch"           , 40000.<m>
             "GPS"                 , 20350.<m>
             "ISS Avg"             ,  382.5<m>
             "Hubble Space Telescope", 595.<m>|] 

let gs = pois |> Array.map (snd >> ( * ) 1000. >> (+) earth_rad >> flipped_field) 
Array.zip pois gs 
|> Array.sortBy snd
|> Array.map (fun ((l,n),g) -> [|l; string n ; string g|])
|> tohtmlTable ["Location"; "Distance in km"; "g"]


GEO Synch,40000,0.184994267215874
GPS,20350,0.557133861020474
~Low Earth Orbit Max,2000,5.67781902995993
Hubble Space Telescope,595,8.19955381460682
ISS Avg,382.5,8.7237513057884
~Low Earth Orbit Min,160,9.32837721530268


In [7]:
//HIDDENINPUT
{Html = """<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/flot/0.8.3/jquery.flot.js"></script>"""}

In [9]:
//HIDDENINPUT [[A script, mix of JS and F# to generate a graph]]
let adjp (s:string) = if s.[..2] = "GEO" then 30 else 0

let poi_annotation adj d g s = 
    let arrow = """ctx.beginPath();ctx.moveTo(lpos.left, lpos.top);ctx.lineTo(lpos.left, lpos.top - 10);ctx.lineTo(lpos.left + 10, lpos.top - 5); ctx.lineTo(lpos.left, lpos.top); ctx.fill();""" 
    
    sprintf
     "$(\"#placeholder\").append(\"<div style='position:absolute;left:\" + (lpos.left + 4) + \"px;top:\" + (lpos.top - %d) + \"px;color:#666;font-size:smaller'>%s (%Akm, %Am/s²)</div>\");%s" adj s d g arrow;

let accels = [for dist in 0.0<m>..1e5<m>..5e7<m> do
                let d = (6.37e6<m> + dist) 
                let g = (earth_grav d 1.<kg>) 
                yield sprintf "[%A,%A]" (roundn 3 (d/1000. - 6370.<m>)) (roundn 3 g)] 

let accels_str = String.Join(",", accels) 

let annots = 
  [for (l,d) in pois -> 
    let g = flipped_field (earth_rad + d * 1000.)
    sprintf "lpos = plot.pointOffset({ x: %A, y: %A});\n%s;"
             d g (poi_annotation (adjp l) (roundn 2 d) (roundn 2 g) l)]
             
let annots_str = "var ctx = plot.getCanvas().getContext(\"2d\");\nvar lpos;\n" + String.Join("\n\n", annots)

let flotTemplate = """
<div id="placeholder" style="width:800px;height:620px;"></div>
<div style='text-align:center'>km</div>
<script type="text/javascript">  

  plot = $.plot("#placeholder", 
   [{ data:  [__ACCEL__], label: "Gravitational Accel = 0.0m" }], 
    { series:{lines:{show: true}},
	  grid:  {hoverable: true},
      yaxis: { tickFormatter: function(val, axis) { return val < axis.max ? val.toFixed(2) : "m/s²";}}
    });

  var legends = $("#placeholder .legendLabel");

  legends.each(function () {
   $(this).css('width', $(this).width());   // fix the widths so they don't jump around
  });

  var updateLegendTimeout = null;
  var latestPosition = null;

  function updateLegend() {

   updateLegendTimeout = null;
   var pos = latestPosition;

   var axes = plot.getAxes();
   if (pos.x < axes.xaxis.min || pos.x > axes.xaxis.max ||
    pos.y < axes.yaxis.min || pos.y > axes.yaxis.max) {
    return;
   }

   var i, j, dataset = plot.getData();
   
   for (i = 0; i < dataset.length; ++i) {
    var series = dataset[i];
    
    for (j = 0; j < series.data.length; ++j) {// Find the nearest points, x-wise
     if (series.data[j][0] > pos.x) {break;}
    }

    // Now Interpolate
    var y,p1 = series.data[j - 1], p2 = series.data[j];

    if (p1 == null) {
     y = p2[1];
    } else if (p2 == null) {
     y = p1[1];
    } else {
     y = p1[1] + (p2[1] - p1[1]) * (pos.x - p1[0]) / (p2[0] - p1[0]);
    }

    legends.eq(i).text(series.label.replace(/=.*/, "= " + y.toFixed(2) + "m/s² (" + pos.x.toFixed(1) +"km)"));
   }
  }
  
  __ANNOTS__

  $("#placeholder").bind("plothover",  function (event, pos, item) {
   latestPosition = pos;
   if (!updateLegendTimeout) {
    updateLegendTimeout = setTimeout(updateLegend, 50);
   }
  }); 
</script>
"""
{Html = flotTemplate.Replace("__ACCEL__",accels_str).Replace("__ANNOTS__", annots_str)}

km

## 2.2 Not Falling is Unnatural

> There is an art, it says, or rather, a knack to flying. The knack lies in learning how to throw yourself at the ground and miss. Pick a nice day, [The Hitchhiker's Guide to the Galaxy] suggests, and try it.

>The first part is easy. All it requires is simply the ability to throw yourself forward with all your weight, and the willingness not to mind that it's going to hurt.

>That is, it's going to hurt if you fail to miss the ground. Most people fail to miss the ground, and if they are really trying properly, the likelihood is that they will fail to miss it fairly hard.

>Clearly, it is the second part, the missing, which presents the difficulties..."

>--Douglas Adams

In [17]:
//HIDDENINPUT
asideHtm "How can those on the ISS feel ~89% of what we get here on the surface at the
same time as experiencing weightlessness? If you're like me, when you asked, you gotten the 
standard \"because they're in free-fall\". I find that explanation lacking; 
it fails to elaborate on what about freefall 
makes one weightless. The <a href='http://physics.stackexchange.com/questions/29929/gravity-on-the-international-space-station'>
clearest analogy</a> I've ever run into links freefall to the above quoted passage 
from THHGTH. Things in orbit around each other are one of the few things able to fall and 
manage to miss the ground. The key to appreciating free-fall is in realizing that things
in orbit and every day falling (ignoring friction and such) are the same thing. 
Unlike how THHGTH recommends acheiving flight however,for things in orbit, it is not so much 
that they miss the ground as it's that the ground keeps moving out of 
the away (it helps to look at it that way instead of as the symmetric situation of 
horizontal vs vertical velocity).
<br/><br/>
To fully understand the phenomenon however, we need disentangle the two meanings of weight 
typically mixed together. There
is weight as in the force exerted by participating in a gravitational field 
and there is the everyday notion of weight
we feel. They're related since our weight comes from being party to a gravitational field 
while something else pushes
back just as hard as the Earth is pulling.<br/><br/>
The problem arises when we combine our notion of fall (downwards) with the idea that weight 
has to feel like
something. But in fact, just being in a (uniform) gravitational field is 
not enough to \"feel solid\", you need
a counter opposing force resisting your inertia. In the everyday world,
this is provided by the Normal Force of say, a chair or the ground
pushing back at you, resisting gravity and keeping you from falling. 

Things that are falling (this includes an apple, a jumper or the moon or the Earth)
have no <i>'felt'</i> weight, the reason the ISS and other objects in orbit manage to achieve 
weightlessness is because the
ground manages to move away before they get a chance to hit it. This is something
we could all manage without having
to move at incredible horizontal velocities if we somehow figured out how to jump
and miss the ground by accident."

How can those on the ISS feel ~89% of what we get here on the surface at the
same time as experiencing weightlessness? If you're like me, when you asked, you gotten the 
standard "because they're in free-fall". I find that explanation lacking; 
it fails to elaborate on what about freefall 
makes one weightless. The 
clearest analogy I've ever run into links freefall to the above quoted passage 
from THHGTH. Things in orbit around each other are one of the few things able to fall and 
manage to miss the ground. The key to appreciating free-fall is in realizing that things
in orbit and every day falling (ignoring friction and such) are the same thing. 
Unlike how THHGTH recommends acheiving flight however,for things in orbit, it is not so much 
that they miss the ground as it's that the ground keeps moving out of 
the away (it helps to look at it that way instead of as the symmetric situation of 
horizontal vs vertical velocity).
 
To fully understand the phenomenon however, we need disentangle the two meanings of weight 
typically mixed together. There
is weight as in the force exerted by participating in a gravitational field 
and there is the everyday notion of weight
we feel. They're related since our weight comes from being party to a gravitational field 
while something else pushes
back just as hard as the Earth is pulling. 
The problem arises when we combine our notion of fall (downwards) with the idea that weight 
has to feel like
something. But in fact, just being in a (uniform) gravitational field is 
not enough to "feel solid", you need
a counter opposing force resisting your inertia. In the everyday world,
this is provided by the Normal Force of say, a chair or the ground
pushing back at you, resisting gravity and keeping you from falling. 

Things that are falling (this includes an apple, a jumper or the moon or the Earth)
have no 'felt' weight, the reason the ISS and other objects in orbit manage to achieve 
weightlessness is because the
ground manages to move away before they get a chance to hit it. This is something
we could all manage without having
to move at incredible horizontal velocities if we somehow figured out how to jump
and miss the ground by accident.

Even basic physics has a lot of ideas that grind against intuition. For example, the idea that objects in motion will remain in motion, unless acted on by a force, is one. The idea that gravity and the concept of weight as we feel it in the everyday is fictious is another counter-intuitive notion. Orbits combine these two. Firstly, because of a relative lack of friction, a large horizontal velocity can last a long time (meanwhile, we are used to large velocities requiring a continuous application of force). An object can continue to miss the ground without any extra expenditure of energy. Together with the idea of weightlessness as the more natural concept for two objects interacting in gravitational field, orbits as falling becomes a bit clearer to grasp (as you fall you follow the curvature of the earth—it also helps to imagine it as an animation, frame by frame; going down the frame makes it look more like falling—almost like we're unrolling the interaction through time).

# 3. Vector Fields

At this point, I can't help but note that I've veered a bit far afield. The original intention of this piece was to connect curried functions from functional programs to fields in physics and yet, here I am talking about how falling is flying with no pesky forces in the way. Nonetheless, I hope the above has been an effective demonstration of the advantage of a computational approach to learning topics commonly thought of as challenging (there'll be more such demonstrations in the below). In reality, much of the difficulty is incidental instead of necessary: for the student, it is grappling with inconsistent and often unmotivated notation, as well as plain general unfamiliarity or counter-intuitiveness and for the teacher: the baggage of being wedded to centuries old tradition of how subjects must be taught. Much pointless complexity arises from the interaction of all those variables.

Right, on topic. The real world has more than just one dimension. Depending on whom you ask, it can be anywhere from 3 or 4 to 11 to 26 or more or less. Most real world (classical) problems settle on 3 however. And here, again we see the advantage of a computational approach. It takes only a few lines to generalize our methodology to vectors (and though it's general to N-dimensions, only for 3—or 2 in a few places—does it really makes sense for the operations we're performing).

In [174]:
//COLLAPSIBLENOOUTPUT [[vector helpers]]
[<FunScript.JS>]
let inline cube r = r * r * r

[<FunScript.JS>]
let inline squared r = r * r

[<FunScript.JS>]
let inline vec_mag (v:float<_> []) = Array.foldBack ((float >> squared >> (+))) v 0. |> sqrt

[<FunScript.JS>]
let sum_force_vecs = Array.fold (Array.map2 (+)) (Array.init 2 (konst 0.<N>))


val inline cube ∈
  r∈ ^a ⟹  ^c
    when  ^a ∈ (static member ( ⨯ ) ∈  ^a ⨯  ^a ⟹  ^b) and
         ( ^b or  ^a) ∈ (static member ( ⨯ ) ∈  ^b ⨯  ^a ⟹  ^c)
val inline squared ∈
  r∈ ^a ⟹  ^b when  ^a ∈ (static member ( ⨯ ) ∈  ^a ⨯  ^a ⟹  ^b)
val inline vec_mag ∈ v∈float<'u> [] ⟹ float
val sum_force_vecs ∈ (float<N> [] [] ⟹ float<N> [])


In [11]:
let grav_field (m1:float<kg>) (r:float<m> []) (m2:float<kg>) =        
    let mag = vec_mag r * 1.<m>
    let scale = (-G * m1 * m2) / cube mag
    r |> Array.map (( * ) scale)


val grav_field ∈ m1∈float<kg> ⟹ r∈float<m> [] ⟹ m2∈float<kg> ⟹ float<N> []


The function is the same as before except that, instead of returning a single number, we now return a list of numbers, representing our force vector. The input for distance has also been replaced with a vector. Below, we apply random masses at different locations to get force vectors. Everything is working correctly.

In [89]:
grav_field earth_mass [|1.<m>; 0.<m>; earth_rad|] 68.0<kg> ,
grav_field earth_mass [|1.<m>; 12.<m>; earth_rad|] 1.0<kg> 


val it ∈ float<N> [] ⨯ float<N> [] =
  ([|-0.0001046461073; 0.0; -666.5957035|],
   [|-1.538913343e-06; -1.846696011e-05; -9.802877992|])

## 3.1 Finally, On Currying vs Fields

The functional way of viewing fields is that they are curried functions; through partial application they can either return more specialized functions or with enough inputs, (e.g.) a vector of forces. For example, the common static field is a function that takes an object of some appropriate type (say, Coulombs or Kg) and returns a function that accepts position vectors which itself returns a function that takes objects (of the same type) and maps them to forces. So by applying (e.g.) a mass, M, to the gravitational field function we specialize to talking about the gravitational field around M. Further specifying a position vector allows us to talk about the force at that distance for various other masses.

In short, fields are really functions. And the way they are used makes them the same as using curried functions with partial application.

### 3.1.1 A quick note on Observables

In quantum mechanics things known as observables can't be avoided. They have an exact mathematical definition as self adjoint operators but for our purposes we can think of them as functions. This is tricky because whereas before, our field function simply took a vector of reals (such as $\mathbb{R}^3$) for position, in quantum mechanics, position is an operator and so is instead something like ($ \mathcal{S} \rightarrow \mathbb{R}$ ). Thinking it through, I realized that the most sensible notion of variables as functions are random variables! A quick [search](https://www.encyclopediaofmath.org/index.php/Quantum_probability#Generalization) reveals that indeed, _"real-valued quantum random variables correspond to self-adjoint operators affiliated with $\mathcal{A}$ [a von Neumann algebra on operators over a Hilbert Space], as postulated in quantum mechanics_". One can also apply the notion of Observables to classical mechanics, and there also, they are functions that smell like random variables. And so, measurement can be thought of as an evaluation and hence, computation. Working backwards and having everything fit this way is really nice.

### 3.1.2 Quantum Field Theory

Quantum Field Theory further muddies our picture because there, people talk about fields as if they were a real thing and particles as excitations in this field. If fields are actually curried functions with specializations obtained through partial application, what does it mean for a function to be excited? However, it's worth remembering that at the bottom of it all, there is a computation that's carried out; a fluctuating field suggests that there's a computation unfolding—in other words, evaluating the field gives different values over time. This is also suggestive of another curiosity—fields in physics are actually two different things. There is the field as function and then there is the underlying computation that is eventually evaluated (or calculation or phenomenon*, the 'real' thing).

In the classical world, measurement yields simple things like vectors, in the quantum realm we get probability distributions. There exist computations whose outputs are also probability distributions: probabilistic programs. Putting all these together, when a physicist says fields are interacting or oscillating, we can think of a probabilistic program (on noncommuting random variables) whose inputs are curried functions and, in some sense, mutually recursive with other 'fields', all of which unfolds some computation (over time).

*_If for whatever reason, you do not like the idea of a computable reality, at the least our only interface with it through testable predictions must be._

### 3.1.3 Speculations or, Here be Dragons

# 4. Examples by Visualizations of and Interacting with Static Fields

In [12]:
//HIDDENINPUT range array
{Html = "<script>
function rangeArray(start,step,end){
    var a = Array()
    for(x=start; x <=end; x+=step)
        a.push(x)
    return a;}
</script>"}

In [13]:
//COLLAPSIBLE [[Visualize Fields]]
[<JSEmitInlineAttribute("rangeArray({0},{1},{2})")>]
let inline rangeArray x y z = [||]

[<JSEmitInlineAttribute("6.674e-11")>]
let getG() = 6.674e-11<N * m^2 / kg^2>

[<FunScript.JS>]
let grav_force (m1:float< kg>) (r:float<m>) (m2:float<kg>) =        
    if r = 0.<m> then 0.<N> 
    else ((m1 * m2) * getG()) / (r * r)
    
[<FunScript.JS>]
let main() =
    
    let mass_moon = 7.3459e22<kg> 
 
    let rad_moon = 1737.5<m> * 1000.
    
    let earth_grav = grav_force 5.96e24<kg>
    
    let earth_rad = 6.37e6<m>

    let jupiter_grav = grav_force 1.89813e27<kg>
    let jupiter_rad = 6.9173e7<m>
    
    let masses = rangeArray 10 1000 100000
    let radii:float<m>[] = rangeArray 0 1000 100000 

    let xss = Array.init masses.Length (fun _ -> Array.copy radii)
    let yss = Array.init masses.Length (fun m -> Array.create radii.Length masses.[m])

    let zss = Array.init masses.Length (fun x ->  
                 Array.init radii.Length (fun y -> 
                  earth_grav (xss.[x].[y] * 1000. + earth_rad) yss.[x].[y]))

    let zss2 = Array.init masses.Length (fun x ->  
                 Array.init radii.Length (fun y -> 
                   jupiter_grav (xss.[x].[y] * 1000. + jupiter_rad) yss.[x].[y]))

    let zss3 = Array.init masses.Length (fun x ->  
                 Array.init radii.Length (fun y -> 
                   grav_force mass_moon (xss.[x].[y] * 1000. + rad_moon) yss.[x].[y]))
    xss,yss,zss,zss2,zss3
                   
Compiler.compileWithoutReturn (<@let xss,yss,zss,zss2,zss3 = main() in ()@>) 
|> embedScript

In [14]:
//HIDDENINPUT include external JS for 3D graphs
{Html = """
<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.js"></script>
	<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r66/three.min.js"></script>
	<script type="text/javascript" src="elegans.min.js"></script> 
"""}

In [15]:
//HIDDENINPUT  Generate Graph
let cols = "{fill_colors:['blue', 'green', 'red','orange','yellow']}"
{Html = String.Format("
<link rel='stylesheet' href='https://rawgit.com/domitry/elegans/master/examples/common.css'>
<div id='visz'>
</div>
<script type='text/javascript'>         
    var data = {{ }};
    var data2 = {{}};
    var data3 = {{}}
    data.x = {0}
    console.log(data.x[0])
    data.y = {1}
    console.log(data.y[0])
    data.z = {2}
    console.log(data.z[0])
    
    data2.x = {0}
    data2.y = {1}
    data2.z = {3}
    
    data3.x = {0}
    data3.y = {1}
    data3.z = {5}
    
    var stage = new Elegans.Stage(d3.select('#visz')[0][0]);
    var sf1 = new Elegans.Surface(data, {4});
    var sf2 = new Elegans.Surface(data2,{4});
    var sf3 = new Elegans.Surface(data3,{4});
    stage.add(sf1);
    stage.add(sf2);
    stage.add(sf3);
    stage.render(); 
</script>","xss", "yss", "zss", "zss2", cols,"zss3")}

In [57]:
//HIDDENINPUT
asHtml """<p style = "border-width:1px;border:solid;border-color:gray;padding:10px">
The graph above is a visualization of a portion of the Moon's, Earth's and Jupiter's gravitational fields. 
It's not the clearest but stacked like this, we can compare them by zooming in and rotating. 
The <b>X</b> axis is the distance from the surface, ranging from <b>0-100,000km</b>. 
The <b>Y's</b> are masses, ranging from <b>10kg - 100,000kg</b>. 
And the <b>Z</b> axis consists of forces felt in <i>Newtons</i>. 
Graphed like this, we can sort of get an idea the shape of the different fields.</p>"""

The graph above is a visualization of a portion of the Moon's, Earth's and Jupiter's gravitational fields. 
It's not the clearest but stacked like this, we can compare them by zooming in and rotating. 
The X axis is the distance from the surface, ranging from 0-100,000km . 
The Y's are masses, ranging from 10kg - 100,000kg . 
And the Z axis consists of forces felt in Newtons . 
Graphed like this, we can sort of get an idea the shape of the different fields.

In [16]:
//HIDDENINPUT
asHtml """<p style="font-size:smaller">The graphing package I used (Elegans), didn't make it 
obvious how to label or format axes and I didn't want to waste time hunting for them. Apologies
for unclear labelling.</p>"""

The graphing package I used (Elegans), didn't make it 
obvious how to label or format axes and I didn't want to waste time hunting for them. Apologies
for unclear labelling.

## 4.1 Generalizing to Static Fields

Looking at the types for a gravitational and electrostatic field makes it clear that both fields can be represented by a single function parameterized by some _Constant_! By looking at the types, the similarity between the two fields is really brought into focus (more so than the similarity between the equations IMO) because the ability to specify both fields with a single function really forces one to take notice of the equivalence.

In [17]:
[<FunScript.JS>]
let static_field (constant:float<'some_constant>)
                 (m1:float<'some_quantity>) 
                 (r:float<m> []) 
                 (m2:float<'some_quantity>) =        
    let mag = vec_mag r * 1.<m>  
    let scale = (constant * m1 * m2) / cube mag
    r |> Array.map (( * ) scale)


val static_field ∈
  constant:float<'some_constant> ⟹
    m1:float<'some_quantity> ⟹
      r:float<m> [] ⟹
        m2:float<'some_quantity> ⟹
          float<'some_constant 'some_quantity²/m²> []

The type tells us that essentially, a static field is _'some_constant * 'some_quantity²/m²_, capturing the inverse square aspect of the relationship. We can then write the gravitational field in terms of static field and all the types work out and the function works as it should:

In [18]:
let gravitational_field (m:float<kg>) (r:float<m> []) (m2:float<kg>) = static_field -G m r m2

gravitational_field earth_mass [|10.<m>; 5.<m>;earth_rad|] 1.<kg>


val gravitational_field ∈
  m∈float<kg> ⟹ r∈float<m> [] ⟹ m2∈float<kg> ⟹ float<N> []
val it ∈ float<N> [] = [|-1.538913343e-05; -7.694566713e-06; -9.802877992|]


In [19]:
[<Measure>] type C

let k = 9e9<N * m ^2 / C^2> 
 
let electric_field (q:float<C>) (r:float<m> []) (q2:float<C>) = static_field k q r q2
                                                                                 
electric_field 1.0<C> [| 1.0<m> |] 1.0<C> = [| 9e9<N> |]


val k ∈ float<N m²/C²> = 9000000000.0
val electric_field ∈ q∈float<C> ⟹ r∈float<m> [] ⟹ q2∈float<C> ⟹ float<N> []
val it ∈ bool = true


In [70]:
G,k, log10 (float k) - log10 (float G)


val it ∈ float<N m²/kg²> ⨯ float<N m²/C²> ⨯ float =
  (6.674e-11, 9000000000.0, 20.12985631)

We can define electric fields in the same manner. From that, it's clear that mass and charge are similar abstractions. The difference between the two fields is that 1) the applied constants have opposite signs and differ by some 20 orders of magnitude, 2) charges are like masses that are allowed to go negative. That opposite attracts is a mere fact of arithmetic, what is more fundamental is that charges are less restricted in that they can be negative.

In [186]:
//HIDDEN Helper functions
[<FunScript.JS>]
module VecHelp = 
    let inline to_cm d = float d * 0.01<m>

    let inline micro_coulombs c = 1e-6<C> * float c

    let inline sub a b = Array.map2 (-) a b

    let inline cm_list l = Seq.map to_cm l |> Seq.toArray

    let inline nano_coulombs c = 1e-9<C> * float c



  val inline to_cm ∈
    d∈ ^a ⟹ float<m> when  ^a ∈ (static member op_Explicit ∈  ^a ⟹ float)
  val inline micro_coulombs ∈
    c∈ ^a ⟹ float<C> when  ^a ∈ (static member op_Explicit ∈  ^a ⟹ float)
  val inline sub ∈
    a∈ ^a [] ⟹ b∈ ^b [] ⟹  ^c []
      when ( ^a or  ^b) ∈ (static member ( - ) ∈  ^a ⨯  ^b ⟹  ^c)
  val inline cm_list ∈
    l∈seq< ^a> ⟹ float<m> []
      when  ^a ∈ (static member op_Explicit ∈  ^a ⟹ float)
  val inline nano_coulombs ∈
    c: ^a ⟹ float<C> when  ^a : (static member op_Explicit :  ^a ⟹ float)
end

In [236]:
//HIDDEN
open VecHelp

In [252]:
//Electrostatic fields
[<FunScript.JS>]
module ElectorstaticFields = 
    let electric_field (q:float<C>) (r:float<m> []) (q2:float<C>) = static_field k q r q2
    
    let eforce_on_q (r,q) (r', q') = electric_field q (sub r r') q'
 
    let efield_force charge qs = qs |> Array.map (eforce_on_q charge) |> sum_force_vecs
    
    let efield_sum qs r = total_efield_force (r,1.0<C>) qs |> Array.map (flip (/) 1.<C>)
   


  val electric_field ∈
    q∈float<C> ⟹ r∈float<m> [] ⟹ q2∈float<C> ⟹ float<N> []
  val eforce_on_q ∈
    r∈float<m> [] ⨯ q∈float<C> ⟹ r'∈float<m> [] ⨯ q'∈float<C> ⟹ float<N> []
  val efield_force ∈
    float<m> [] ⨯ float<C> ⟹ qs∈(float<m> [] ⨯ float<C>) [] ⟹ float<N> []
  val efield_sum ∈
    qs:(float<m> [] ⨯ float<C>) [] ⟹ r:float<m> [] ⟹ float<N/C> []
end

At this point, it's instructive to highlight an example of the unnecessary obtuseness that abounds in physics equations. Given some test charge q, and a set of discrete charges, the electric field felt by q is given by: $$F(r) = q \cdot k \sum\limits_{i=1}^n {q_i\frac{r-r_i}{|r-r_i|^3}}$$

The problem of course, is that the Equation does not make clear that the function $F$ should take a pair of variables: the charge and its vector. It's assumed implicity, and thus one more thing to be confused about. The same function is given by `efield_force` where the type makes it clear that the field is parameterized by both a position vector _and_ a charge.

In [229]:
//HIDDEN
open Prelude.Math
let rad2deg r = r * 180./pi 
let clampAngle r = let a = r % (2. * pi) in if a < 0. then a + 2. * pi else a


val rad2deg ∈ r∈float ⟹ float
val clampAngle ∈ r∈float ⟹ float


In [178]:
//HIDDEN wolfram alpha, prelude
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2012\Projects\prelude\prelude\bin\release\prelude.dll"
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2012\open\FSharp.Data.2\bin\Fsharp.Data.dll"

#time "on"
   
open FSharp.Data
open System 
open Prelude.Common 

type Wolfram = FSharp.Data.XmlProvider<"""C:\Users\sir.deenicus\Documents\wolfram.xml"""> 
let wc = new Net.WebClient()
let queryWolfram q = 
    let url = urlencode q
    let r = wc.DownloadData("http://api.wolframalpha.com/v2/query?appid=QJ8478-EVP62RGYVR&input="+url+"&format=plaintext")
    let w = Wolfram.Parse (r |> String.DecodeFromUtf8Bytes)
            
    w.Pods |> Array.map (fun p -> 
                p.Title, 
                p.Subpods |> Array.map (fun sp -> sp.Plaintext)) 
             |> Map.ofArray
                                       


val wc ∈ Net.WebClient = System.Net.WebClient
val queryWolfram ∈ q∈string ⟹ Map<string,string []>


In [ ]:
click field
animate gravity
electric and grave of proton
negative mass

In [113]:
let mass_inf, c_inf = queryWolfram "mass of proton to kg", queryWolfram "charge of proton in Coulombs"
mass_inf.["Result"], c_inf.["Result"]


val mass_inf ∈ Map<string,string []> =
  map
    [("Additional conversion", [|"1.672622×10^-24 grams"|]);
     ("Comparisons as mass",
      [|" ≈ 0.5 × deuteron mass (≈ 3.3×10^-27 kg )";
        " ≈ 0.53 × tau particle mass (≈ 1777 MeV/c^2 )";
        " ≈ 1.007 × unified atomic mass unit ( 1.660539×10^-27 kg )"|]);
     ("Corresponding quantities",
      [|"Relativistic energy E from Emc^2∈
  | 938 MeV  (megaelectronvolts)";
        "Characteristic length L from Lh/(mc)∈
  | 1.3 fm  (femtometers)";
        "Thermal de Broglie wavelength at 100 K from λh/(2πmkT)^(1/2)"+[26 chars];
        "Characteristic time T from Th/(mc^2)∈
  | 4.4×10^-24 seconds";
        "Thermodynamic temperature T from kTmc^2∈
  | 1.089×10^13 K  (kelvins)";
        "Compton frequency ν from νmc^2/h∈
  | 2.269×10^23 Hz  (hertz)"|]);
     ("Input interpretation", [|"convert p  (proton) | mass to kilograms"|]);
     ("Result", [|"1.672622×10^-27 kg  (kilograms)"|])]
val c_inf ∈ Map<string,string []> =
  map


In [255]:
open ElectorstaticFields

efield_sum charge_density (cm_list [|1.; sqrt 3.|]) //|> Array.map ((*) (-2. |> nano_coulombs))


val it ∈ float<N/C> [] = [|0.0; 38971.14317|]


In [274]:
open ElectorstaticFields
[for x in 0.0..9. do
 for y in 0.0..9. ->
    Array.append [|string x; string y|] (efield_sum charge_density (cm_list [|x; y|]) |> Array.map string)]
 |> tohtmlTable []

0,0,NaN,NaN
0,1,-16099.6894379985,98049.8447189992
0,2,-7954.95128834866,30454.9512883487
0,3,-3840.23212771313,15760.3481915697
0,4,-2012.46117974981,9649.92235949962
0,5,-1152.59175420239,6481.47938550598
0,6,-711.512473537886,4634.53742061366
0,7,-466.507575674793,3469.51120873933
0,8,-321.003033136323,2690.26213254529
0,9,-229.691072984459,2144.72093954118
1,0,0,0


In [160]:
[<FunScript.JS>]
let mainc() =
    let x,y = 2.,3.
   
    vec_mag [|x;y|],electric_field_sum charge_density (cm_list [|x; y|]) |> vec_mag

let js2 = Compiler.compileWithoutReturn (<@mainc()@>)
js2


val mainc ∈ unit ⟹ float ⨯ float
val js2 ∈ string =
  "var TupleDouble____Double, TupleDouble_Double, MathJS__Sqrt$,"+[14860 chars]
val it ∈ string =
  "var TupleDouble____Double, TupleDouble_Double, MathJS__Sqrt$,"+[14860 chars]

In [174]:
//HIDDEN
let js3 = Compiler.compileWithoutReturn (<@ let tarr = Array.init 50 (fun _ -> Array.create 50 1) in Globals.console.log tarr @>)

{Html="<script>" + js3 + "</script>"}

In [161]:
//HIDDEN

js2



val it ∈ string =
  "var TupleDouble____Double, TupleDouble_Double, MathJS__Sqrt$, FSI_0207__mainc$, FSI_0206__get_charge_density$, FSI_0206__electric_field_sum$, FSI_0206__electric_field_at_r$, FSI_0206__charge_density, FSI_0197__vec_mag$, FSI_0197__sum_force_vecs, FSI_0197__squared$Double__Double_Double_Double, FSI_0197__get_sum_force_vecs$, FSI_0196__konst$Double__Int32_Double_Int32, FSI_0186__to_cm$Double_Double, FSI_0186__sub$Double__Double__Double_Double_Double_Double, FSI_0186__micro_coulombs$Int32_Int32, FSI_0186__micro_coulombs$Double_Double, FSI_0186__cm_list$Double_Double, FSI_0179__electric_field$, FSI_0176__k, FSI_0172__static_field$, FSI_0113__vec_mag$, FSI_0113__squared$Double__Double_Double_Double, FSI_0113__cube$Double__Double__Double_Double_Double_Double, Array__ZeroCreate$Tuple_2_Double____Double_Tuple_2_Double____Double_, Array__ZeroCreate$Double___Double___, Array__ZeroCreate$Double_Double, Array__MapIndexed2$Double__Double__Double_Double_Double_Double, Array__Map

In [24]:
//COLLAPSIBLENOOUTPUT [[Funscript Canvas]]
[<FunScript.JS>]
let main () =   
    let canvas = Globals.document.getElementById("canv") :?> HTMLCanvasElement 

    let ctx = canvas.getContext_2d()      

    let drawArrowHead(x1,y1,x2,y2) =     
      
      let rad = (atan2 (y2-y1) (x2-x1)) + (if x2>=x1 then 1. else -1.)*System.Math.PI/2.;
       
      ctx.save();
      ctx.beginPath();
      ctx.translate(x2,y2);
      ctx.rotate(rad);
      ctx.moveTo(0.,0.);
      ctx.lineTo(5.,8.);
      ctx.lineTo(-5.,8.);
      ctx.closePath();
      ctx.restore();
      ctx.fill();
      ()

    let drawLine(x1,y1,x2,y2,col:string) =
      ctx.strokeStyle <- col;
      ctx.fillStyle <- col;
      ctx.lineWidth <- 1.;

      // draw the line
      ctx.beginPath();
      ctx.moveTo(x1,y1);
      ctx.lineTo(x2,y2);
      ctx.stroke();
      (x1,y1,x2,y2)

    let getMousePos(e:MouseEvent) =
        let rect = canvas.getBoundingClientRect();
        e.clientX - rect.left, e.clientY - rect.top

    let drawCircle(x,y) =
        ctx.beginPath()
        ctx.arc(x,y,5.,0.,2. * System.Math.PI);
        ctx.fill()
        ctx.stroke()
        ()

    let infdiv = Globals.document.getElementById("infdiv")

    canvas.addEventListener_mousemove(fun e ->  
        let x,y = getMousePos e 
        infdiv.innerHTML <- x.ToString() + ", " + y.ToString()
        null)  
    
    let drawArrow = drawLine >> drawArrowHead
    
    canvas.addEventListener_mousedown(fun e ->   
        drawCircle(getMousePos e); 
        let x,y = getMousePos e
        drawArrow(0.,0.,x,y,"rgba(0,0,255,0.3)")
        drawArrow(x,y, x + 50.,y+20.,"red")
        null)      
    ()
      
let js = Compiler.compileWithoutReturn (<@main()@>)  //(<@ MyNamespace.MyModule.SomeFunc @>)

let html = """
<div id="infdiv" style="width:100px;height:100px"></div>
<canvas id="canv" width="300" height="300" style="border-width:1px;border-style:solid"></canvas>

"""




val main ∈ unit ⟹ unit
val js ∈ string =
  "var TupleDouble_Double_Double_Double_String, TupleDouble_Doub"+[8176 chars]
val html ∈ string =
  "
<div id="infdiv" style="width:100px;height:100px"></div>
<ca"+[95 chars]

In [103]:
//HIDDEN
html + "<script>" + js + "</script>"


val it ∈ string =
  "
<div id="infdiv" style="width:100px;height:100px"/>
<canvas id="canv"" width="300" height="300" style="border-width:1px;border-style:solid"/>
<script>var TupleDouble_Double_Double_Double, TupleDouble_Double, MathJS__Atan2$, LanguagePrimitives__UnboxGeneric$HTMLCanvasElement_HTMLCanvasElement_, FSI_0151__main$;
  FSI_0151__main$ = (function (unitVar0)
  {
    var canvas = LanguagePrimitives__UnboxGeneric$HTMLCanvasElement_HTMLCanvasElement_(((window.document).getElementById("canv")));
    var ctx = (canvas.getContext("2d"));
    var drawArrowHead = (function (tupledArg)
    {
      var x1 = tupledArg.Items[0.000000];
      var y1 = tupledArg.Items[1.000000];
      var x2 = tupledArg.Items[2.000000];
      var y2 = tupledArg.Items[3.000000];
      var _17;
      var _27;
      var _28;
      var _29;
      if ((x2 > x1)) 
      {
        _29 = 1.000000;
      }
      else
      {
        _29 = -1.000000;
      };
      _28 = (_29 ⨯ 3.141593);
      _27 = (_28 / 2.0

In [25]:
{Html = html + "<script>" + js + "</script>"}

In [259]:
//HIDDEN csquery
//
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2012\open\CsQuery\distribution\csquery.dll"
open System

In [293]:
//HIDDEN Clean notebook
//METAIGNORE
let gethtml f s = s |> Seq.toArray |> Array.map f   

let cq = CsQuery.CQ.Create(IO.File.ReadAllText(@"C:\Users\sir.deenicus\Documents\Fields as Curried Functions.html"))

let divs = cq.Select(".input")

let hiddens = divs.Filter(fun div ->
    div.InnerHTML.Contains "HIDDEN" && 
    (not (div.InnerHTML.Contains "HIDDENINPUT") || div.InnerHTML.Contains "METAIGNORE")).Parent()

hiddens.Remove()

let hiddenins = divs.Filter(fun div -> div.InnerHTML.Contains "HIDDENINPUT")
hiddenins.Next().Find(".prompt").Each(fun n -> n.InnerText <- "")
hiddenins.Remove()

let toggleScript = CsQuery.CQ.Create ("<script>
function toggleHeader(div) {

    var nextSibling = div.nextSibling;
    var isVisible = nextSibling.offsetWidth > 0 || nextSibling.offsetHeight > 0;

    if(isVisible){
      nextSibling.style.display = 'none';
      div.innerHTML = 'Expand ' + div.getAttribute('info');}
    else {nextSibling.style.display = '';
          div.innerHTML = 'Collapse ' + div.getAttribute('info');}
}
</script>")

let scripts = cq.Select("head").Find("script")  

toggleScript.InsertAfter(scripts.Last())
 
///////////////

let collapsibles = divs.Filter(fun div -> 
    div.InnerHTML.Contains "COLLAPSIBLE" && not (div.InnerHTML.Contains "METAIGNORE"))

collapsibles.Each(fun n -> n.Style.SetStyle("display", "none"))

let headers = 
   collapsibles |> gethtml (fun n -> 
      let txt = n.InnerText
      let i0 = txt.IndexOf("[[")
      if i0 <> -1 && i0 < 50 then
         let i2 = txt.IndexOf("]]")
         txt.[i0+2..i2-1]
      else "") |> Array.filter ((<>) "")

let toggleheader = CsQuery.CQ.Create ("<div class = 'collheader' onclick='toggleHeader(this)' style='background-color:gray;cursor:pointer;margin-left:83.5px;color:white;font-weight:bold;font-size:18px'>REPLACE</div>")

toggleheader.InsertBefore collapsibles
 
divs.Select(".collheader").Each(fun i n -> 
    n.InnerHTML <- "Expand " + headers.[i]; n.SetAttribute("info", headers.[i]))

let no_outs = collapsibles.Filter(fun div -> div.InnerHTML.Contains "COLLAPSIBLENOOUTPUT")
no_outs.Next().Remove() 

let hide_outs = divs.Filter(fun div -> div.InnerHTML.Contains "HIDEOUTPUT")
hide_outs.Next().Remove() 

let nhtm = cq.Render()

IO.File.WriteAllText(@"C:\Users\sir.deenicus\Anaconda\Fields_as_Curried_Functions.html", nhtm)


val gethtml ∈ f∈('a ⟹ 'b) ⟹ s∈seq<'a> ⟹ 'b []
val cq ∈ CsQuery.CQ
val divs ∈ CsQuery.CQ
val hiddens ∈ CsQuery.CQ
val hiddenins ∈ CsQuery.CQ
val toggleScript ∈ CsQuery.CQ
val scripts ∈ CsQuery.CQ
val collapsibles ∈ CsQuery.CQ
val headers ∈ string [] =
  [|"vector helpers"; "Visualize Fields"; "Funscript Canvas"|]
val toggleheader ∈ CsQuery.CQ
val no_outs ∈ CsQuery.CQ
val hide_outs ∈ CsQuery.CQ
val nhtm ∈ string =
  "<!DOCTYPE html><html><head>
<meta charset="utf-8">
<title>Fi"+[319830 chars]
val it ∈ unit = ()
